In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import OrderedDict
def get_sports():
    df_sport_latest = pd.read_csv('articles/sports_articles.csv', encoding = "ISO-8859-1")
    df_sport_latest_tv2 = pd.read_csv('articles/sports_articles_tv2.csv', encoding = "ISO-8859-1")
    df_sport_2019 = pd.read_csv('articles/sports_articles_2019.csv', encoding = "ISO-8859-1")
    df_sport_2020 = pd.read_csv('articles/sports_articles_2020.csv', encoding = "ISO-8859-1")
    df_sport_2022 = pd.read_csv('articles/sports_articles_2022.csv', encoding = "ISO-8859-1")
    df = pd.concat([df_sport_latest, df_sport_latest_tv2, df_sport_2019, df_sport_2020, df_sport_2022])
    df = df.sample(frac=1).reset_index(drop=True)
    return df


In [ ]:
def vocab_2_pdset(columns, df):
    df_vocab_select_columns = df.iloc[:, columns]
    vocab_all_values = df_vocab_select_columns.values.ravel()
    return set(vocab_all_values)

def vocab_2_dict(sets):
    assert(len(sets) == 4)
    word_set = sets[0].union(sets[1],sets[2], sets[3])
    df = pd.DataFrame(list(word_set), columns=["Words"])
    df.sort_values(by="Words", inplace=True)
    df.reset_index(drop=True, inplace=True)
    return OrderedDict.fromkeys(word_set)

def get_vocab_dict():
    df_ods_vocab = pd.read_table('ods_fullforms_2020-08-26.csv', header=None)
    df_ddo_vocab = pd.read_table('ddo_fullforms_2020-08-26.csv', header=None)
    df_vocab = pd.read_table('cor1.02.tsv', header=None)
    df_sport_lingo = pd.read_table('sport_lingo.csv', header=None)

    vocab_set = vocab_2_pdset([1,3], df_vocab)
    ods_vocab_set = vocab_2_pdset([0,1], df_ods_vocab)
    ddo_vocab_set = vocab_2_pdset([0,1], df_ddo_vocab)
    sport_lingo_set = vocab_2_pdset([0], df_sport_lingo)

    return vocab_2_dict([vocab_set, ods_vocab_set, ddo_vocab_set, sport_lingo_set])

ordered_dict = get_vocab_dict()


In [ ]:
df_sport = get_sports()

df_sport_combined = df_sport.copy().drop('Link', axis=1)
df_sport_combined.to_csv('articles_temp/combined.csv')


duplicate_rows = df_sport.duplicated()

print(df_sport[duplicate_rows])




In [ ]:
print(df_sport_combined.iloc[:1])

In [ ]:
import time
# isin_dict = False
def test_lookup_performance():
    word_to_check = "linebreak"
    start_time = time.time()

    for x in range(1000000):
        isin_dict = word_to_check in ordered_dict

    end_time = time.time()  
    assert(end_time - start_time < 1)
    print(isin_dict)

test_lookup_performance()

# isin_dict


In [ ]:
import re

train_text = df_sport.iloc[:, [0,1,2]].apply(' '.join, axis=1).replace('\xa0', '', regex=True).to_numpy()

words_arr = []
max_words_insentence = 0

def replace_digits(word):
    return re.sub(r'\d+', 'X', word)

def remove_specials(word):
    words_new = []
    parts = re.findall(r"[A-ZÆØÅa-zæøå0-9]+|\S", word)
    words_new.extend([x for x in parts])
    return words_new

def contains_non_alphanumeric(word):
    return bool(re.search(r'[^a-zæøåA-ZÆØÅ0-9]', word))

def formatWord(word):
    if any(char.isdigit() for char in word):
        return replace_digits(word)
    
    
for ind, sentence in enumerate(train_text):
    sentence_trimmed = sentence.strip()
    words = sentence_trimmed.split()
    current_count = len(words_arr)
    for word in words:
        w = remove_specials(word)
        words_arr.extend([x.lower() for x in w])

    after_count = len(words_arr) - current_count
    if after_count > max_words_insentence:
        max_words_insentence = after_count


words_sport_unique = set(words_arr)
words_sport_unique_list = list(words_sport_unique)
words_sport_lingo = []
words_train_vocab = []

# TODO : brug tensorflow Tokenezier til at omdanne ord til tokens
# TODO : søg i alle leksikoner, søg med og uden bindestreg
# TODO : håndter tal ikke i ordbøger eks ( x-x eller x-årig)
# TODO : lemmatizer : udelad bøjninger af samme navneord. eks : verdensmester/verdensmesteren
# TODO : evt. grupper ord der ofte hænger sammen med nltk BigramFinder. eks vandt over

for w in range(len(words_sport_unique_list)):
    word = words_sport_unique_list[w]
    if any(char.isdigit() for char in word):
        words_train_vocab.append(word)
    if contains_non_alphanumeric(word):
        words_train_vocab.append(word)
    else: 
        isin_dict = word in ordered_dict
        if (isin_dict == False):
            words_sport_lingo.append(word)
        else:
            words_train_vocab.append(word)


print("Max words in sentence: ", max_words_insentence)
print("total unique words:", len(words_sport_unique) )
print("total sports lingo words:", len(words_sport_lingo) )
print("total vocab:", len(words_train_vocab))
print("total articles:", len(df_sport) )


In [ ]:
# def remove_stopwords(word):
#     if word in da_stopwords:
#         return True
#     else:
#         return False

# words_train_vocab = [x for x in words_train_vocab if not remove_stopwords(x)]

# words_train_vocab


In [ ]:
# from collections import Counter
# cnt = Counter()
# for word in words_train_vocab:
#    cnt[word] += 1

# s = sorted(cnt.items(), key=lambda item: item[1])
# s.reverse()

# s


In [ ]:
words_sport_lingo
file = open('words_sport_lingo.txt','w')
for item in words_sport_lingo:
	file.write(item+"\n")
file.close()


file = open('words_train_vocab.txt','w')
for item in sorted(words_train_vocab):
	file.write(item+"\n")
file.close()

In [ ]:
def format_2_bool(x):
    if type(x) == bool:
        return x
    assert(type(x) == str)
    x_copy = x
    x_copy = x_copy.strip()
    x_copy = x_copy.lower()
    assert(x_copy == "true" or x_copy == "false")
    if x_copy == "true":
        return True
    else:
        return False

In [ ]:

df_sport_labels = df_sport['isResult'].apply(lambda x: format_2_bool(x))

results_true = df_sport_labels.loc[df_sport_labels== True]
results_false = df_sport_labels.loc[df_sport_labels == False]

assert(len(results_true) + len(results_false) == len(df_sport_labels))

print(len(results_true))
print(len(results_false))
labels = df_sport_labels.to_numpy().astype(int)

In [ ]:
print(len(labels))
print(len(train_text))
print(labels.shape)
print(train_text.shape)
longest_text = len(max(train_text, key=len))
print("longest text: ", longest_text)
train_text[0]

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

# TODO : evt indikere hvilke navneord der starte med stort bogstav(egenavne), evt. lave et opslag for at undersøge ordklasse for det første ord i sætningen 
# TODO : adskil og inkluder punktuering (,.""?) 

def to_lower(word):
    return tf.strings.lower(word, encoding='utf-8')

def split_specials(input_data):
    fd = [":" , "/", ":", ",", "'", ".", "?", "-", "!", "(", ")", '"']

    new_str = input_data
    for sign in fd:
        r = "\\" + sign
        new_str = tf.strings.regex_replace(new_str, pattern=r, rewrite=" " + sign + " ")

    return new_str

def replace_digits(word):
    return tf.strings.regex_replace(word, pattern=r'\d+', rewrite=r'X')


def custom_standardization(input_data):
    lowercase = to_lower(input_data)
    replaced_digits = replace_digits(lowercase)
    return split_specials(replaced_digits)


# Model constants.
max_features = 3300
sequence_length = 100

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)


text_ds = vectorize_layer.adapt(words_train_vocab)
vect_vocab = vectorize_layer.get_vocabulary()

def vect_layer_2_text(vect_l):
    return np.array([vect_vocab[x] for x in np.trim_zeros(np.squeeze(vect_l.numpy()))])

print("Total vocab/max_features : ",  len(vect_vocab))




In [ ]:
for t in train_text[0:10]:
    print("Original \n:", t)
    print("vect_2_text: \n", vect_layer_2_text(vectorize_layer([t])))
    print("\n")


In [ ]:
def split_data(data, labels, percentage):
    l = len(data)
    p = l - int((percentage/100) * l)
    return (data[0:p], data[p:], labels[0:p], labels[p:])



train_data, val_data, train_labels, val_labels = split_data(vectorize_layer(train_text), labels, 10)

print("Total data points", len(train_text))
print("Train data length", len(train_data))
print("Valkidation data length", len(val_data))


In [ ]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions



In [ ]:
from tensorflow.keras import layers
import random as python_random



def get_transformer_model():

    embed_dim = 96  # Embedding size for each token
    num_heads = 3  # Number of attention heads
    ff_dim = 32  # Hidden layer size in feed forward network inside transformer


    # A integer input for vocab indices.
    inputs = tf.keras.Input(shape=(sequence_length,), dtype="int64")

    # Next, we add a layer to map those vocab indices into a space of dimensionality
    # 'embedding_dim'.
    # x = layers.Embedding(max_features, embed_dim)(inputs)

    embedding_layer = TokenAndPositionEmbedding(sequence_length, max_features, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    # x = layers.Dropout(0.1)(x)

    # Conv1D + global max pooling
    # x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)
    # x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)


    x = layers.GlobalMaxPooling1D()(x)

    # We add a vanilla hidden layer:
    x = layers.Dense(128, activation="relu")(x)
    # x = layers.Dropout(0.5)(x)

    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)


    transformer_model = tf.keras.Model(inputs, predictions)

    # Compile the model with binary crossentropy loss and an adam optimizer.
    transformer_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return transformer_model


In [ ]:
get_transformer_model()

In [ ]:
from tensorflow.keras import layers
import random as python_random

def get_cnn_model():

    embedding_dim = 96

    # A integer input for vocab indices.
    inputs = tf.keras.Input(shape=(None,), dtype="int64")

    # Next, we add a layer to map those vocab indices into a space of dimensionality
    # 'embedding_dim'.
    x = layers.Embedding(max_features, embedding_dim)(inputs)
    x = layers.Dropout(0.5)(x)

    # Conv1D + global max pooling
    x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)
    x = layers.Conv1D(128, 10, padding="valid", activation="relu", strides=3)(x)


    x = layers.GlobalMaxPooling1D()(x)

    # We add a vanilla hidden layer:
    x = layers.Dense(128, activation="relu")(x)
    # x = layers.Dropout(0.5)(x)

    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)
    cnn_model = tf.keras.Model(inputs, predictions)

    # Compile the model with binary crossentropy loss and an adam optimizer.
    cnn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return cnn_model



In [ ]:

def prepare_model(name):
    if (name == "cnn"):
       return get_cnn_model()
    elif (name == "transformer"):
       return get_transformer_model()
  

def filter_max_accuracy(history, threshold = 0.99):
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    list = []
    for x in range(len(acc)):
        if (acc[x] > threshold):
            list.append(val_acc[x])

    return np.array(list)

models = ["cnn", "transformer"]


def mean_model_accuracy(mode_names, iterations, epochs = 20):

    callback_3_loss = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4)

    results = []

    for name in range(len(mode_names)):
        model_name = mode_names[name]
        val_accuracies = []
        
        for x in range(iterations):
            model = prepare_model(model_name)

            # Fit the model using the train and test datasets.
            history = model.fit(train_data, train_labels, epochs=epochs, batch_size=6, validation_data=(val_data, val_labels), callbacks=[callback_3_loss])

            max_val_acc = filter_max_accuracy(history)
            val_accuracies.append(max(max_val_acc))
            print(max(max_val_acc))
            print(val_accuracies)
        
        d = dict(name = model_name, results = np.mean(np.squeeze(np.array(val_accuracies))))
        results.append(d)
        
    return results


In [ ]:
mean_results = mean_model_accuracy(models, 10)
mean_results

In [ ]:
def print_model_score(model):
    score = model.evaluate(val_data, val_labels, verbose=0)
    print("Validation loss:", score[0])
    print("Validations accuracy:", score[1])

def print_validation_results(predictions):
    for x in range(len(val_data)):
        print("VALIDATION SAMPLE: \n" ,vect_layer_2_text(val_data[x]))
        print("LABEL --:" , val_labels[x])
        print("PREDICTION --:" , predictions[x])
        print("\n")

In [ ]:

epochs= 5
transformer_model = get_transformer_model()

# # Fit the model using the train and test datasets.
transformer_history = transformer_model.fit(train_data, train_labels, epochs=epochs, batch_size=6, validation_data=(val_data, val_labels))

In [ ]:

epochs= 5
cnn_model = get_cnn_model()

# # Fit the model using the train and test datasets.
transformer_history = cnn_model.fit(train_data, train_labels, epochs=epochs, batch_size=6, validation_data=(val_data, val_labels))

In [ ]:
def print_results(model):
    np.set_printoptions(precision = 5, suppress = True)
    predictions = model.predict(val_data)
    print_model_score(model)
    print("\n")
    print_validation_results(predictions)
  

In [ ]:
print("--- TRANSFORMER ---")
print_results(transformer_model)

In [ ]:

print("--- CNN ---")
print_results(cnn_model)

In [ ]:
import os

# Set up a logs directory, so Tensorboard knows where to look for files.

ll = transformer_model.layers[1]
ll_weights = ll.get_weights()[0]

print(ll_weights.shape)
ll_weights


In [ ]:
##import I/O module in python
import io

##open the text stream for vectors
vectors = io.open('vectors.tsv', 'w', encoding='utf-8')

##open the text stream for metadata
meta = io.open('meta.tsv', 'w', encoding='utf-8')


##write each word and its corresponding embedding
for index in range(1, len(vect_vocab)):
  word = vect_vocab[index]  # flipping the key-value in word_index
  embeddings = ll_weights[index]
  meta.write(word + "\n")
  vectors.write('\t'.join([str(x) for x in embeddings]) + "\n")

##close the stream
vectors.close()
meta.close()

In [ ]:
# from nltk import collocations
# bigram_measures = collocations.BigramAssocMeasures()
# finder = collocations.BigramCollocationFinder.from_words(["New", "York", "is", "big", "New", "York", "is", "dirty"])
# finder.ngram_fd.items()



In [ ]:
# import lemmy
# # Create an instance of the standalone lemmatizer.
# lemmatizer = lemmy.load("da")

# # Find lemma for the word 'akvariernes'. First argument is an empty POS tag.
# lemmatizer.lemmatize("NOUN", "storsejr")



In [ ]:
# import nltk as nltk
# # from string import punctuation
# # from nltk.corpus import stopwords
# # nltk.download('stopwords')

# # da_stopwords = stopwords.words("danish")


In [ ]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = transformer_model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="transformer_crossentropy", optimizer="adam", metrics=["accuracy"]
)


In [ ]:
print("\nSkisport, Results: ")
print( end_to_end_model.predict(
    [
    "Skisport Buller overrasker alle og gør det umulige. Ny rekord sikrer flerer medalje til Norge",
    "Skisport Buller overrasker alle. Hollænderen havde masser af overskud, da han kom i mål som nummer to",
     ]))

print("\nSkisport, other: ")

print( end_to_end_model.predict(
    [
    "Skisport Sverige drømmer om flere medaljer og sejre til næste års OL. Træner har et godt øje til hidtil ukendt talent",
    "Skisport Flere forventer podiepladser til Norge ved kommende VM. Træner har et godt øje til hidtil ukendt talent",
    "Skisport Denne gang er det alvor. Buller: 'Jeg har satset alt'",
     ]))

In [ ]:


print("\nFodbold, Results:")

print(end_to_end_model.predict(
    [
      "Fodbold Fjerritslev vinder over Vordingborg. Træner kommenterer på historisk kamp",

     ]))

print("\nFodbold, other:") 
print(end_to_end_model.predict(
    [
      "Fodbold Træner forventer at Fjerritslev vinder over Vordingborg. 'Det bliver en historisk kamp'",
     ]))